In [12]:
from pettingzoo.mpe import simple_adversary_v3

env = simple_adversary_v3.parallel_env(render_mode="human")
observations, infos = env.reset()
print('observations:',observations)
print('infos:',infos)
print('agents:',env.agents)
action = {agent: env.action_space(agent).sample() for agent in env.agents}
print('action:',action)
observations, rewards, terminations, truncations, infos = env.step(action)
print('observations:',observations)
print('rewards:',rewards)
print('terminations:',terminations)
print('truncations:',truncations)
print('infos:',infos)
env.close()
print('state_dim:',env.observation_space('agent_0').shape[0])
print('action_dim:',env.action_space('agent_0').n)   #env.action_spaces['agent_0'].n) 是个字典
print('max_num_agents:',env.max_num_agents)
print('n_agents:',env.num_agents)

observations: {'adversary_0': array([-0.32956174, -1.108241  , -0.7522789 ,  0.12979814, -0.35218278,
       -1.3779373 , -0.8661719 , -1.4783411 ], dtype=float32), 'agent_0': array([-0.40009615,  1.5077355 ,  0.02262103,  0.26969635, -0.40009615,
        1.5077355 ,  0.35218278,  1.3779373 , -0.51398915, -0.10040382],
      dtype=float32), 'agent_1': array([0.11389301, 1.6081393 , 0.5366102 , 0.37010017, 0.11389301,
       1.6081393 , 0.8661719 , 1.4783411 , 0.51398915, 0.10040382],
      dtype=float32)}
infos: {'adversary_0': {}, 'agent_0': {}, 'agent_1': {}}
agents: ['adversary_0', 'agent_0', 'agent_1']
action: {'adversary_0': 4, 'agent_0': 2, 'agent_1': 1}
observations: {'adversary_0': array([-0.32956174, -1.108241  , -0.7522789 ,  0.12979814, -0.35218278,
       -1.3779373 , -0.8661719 , -1.4783411 ], dtype=float32), 'agent_0': array([-0.40009615,  1.5077355 ,  0.02262103,  0.26969635, -0.40009615,
        1.5077355 ,  0.35218278,  1.3779373 , -0.51398915, -0.10040382],
      dtyp

In [20]:
from MADDPG import MADDPG
import torch
from pettingzoo.mpe import simple_adversary_v3, simple_spread_v3, simple_tag_v3
def get_env(env_name, ep_len=25):
    """create environment and get observation and action dimension of each agent in this environment"""
    new_env = None
    if env_name == 'simple_adversary_v3':
        new_env = simple_adversary_v3.parallel_env(max_cycles=ep_len,render_mode='rgb_array')
    if env_name == 'simple_spread_v3':
        new_env = simple_spread_v3.parallel_env(max_cycles=ep_len)
    if env_name == 'simple_tag_v3':
        new_env = simple_tag_v3.parallel_env(max_cycles=ep_len)

    new_env.reset()
    _dim_info = {}
    for agent_id in new_env.agents:
        _dim_info[agent_id] = []  # [obs_dim, act_dim]
        _dim_info[agent_id].append(new_env.observation_space(agent_id).shape[0])
        _dim_info[agent_id].append(new_env.action_space(agent_id).n)

    return new_env, _dim_info
dim_info = get_env('simple_adversary_v3')[1]
#dim_info, capacity, batch_size, actor_lr, critic_lr, res_dir)
maddpg = MADDPG(dim_info,1,36,0.01,0.01,'test')

print(maddpg.select_action(observations))

{'adversary_0': 1, 'agent_0': 0, 'agent_1': 1}


In [1]:
from collections import defaultdict

# 创建一个 defaultdict，默认值类型为 int
dd = defaultdict(int)

# 向 defaultdict 中添加键值对
dd['adversary_0'] = -0.2605955550976603
dd['agent_0'] = -0.8693780243973747
dd['agent_1'] = -0.8693780243973747

# 打印 defaultdict
print(dd)


defaultdict(<class 'int'>, {'adversary_0': -0.2605955550976603, 'agent_0': -0.8693780243973747, 'agent_1': -0.8693780243973747})


In [9]:
from Agent import Agent
import torch

agent = Agent(12,1,36,0.01,0.01)
obs = torch.tensor([1,2,3,4,5,6,7,8,9,10,11,12],dtype=torch.float32)
print(agent.action(obs))
print(agent.target_action(obs))


tensor([1.], grad_fn=<AddBackward0>)
tensor(1.)


In [4]:
import math
print(math.cos(0))
print(math.cos(math.pi/2))
print(math.cos(math.pi))
print(math.cos(3*math.pi/2))
print(math.cos(2*math.pi))

print(math.tanh(0))

1.0
6.123233995736766e-17
-1.0
-1.8369701987210297e-16
1.0
0.0007963267107332633


In [8]:
import numpy as np
action =1
action = action + 0.1* np.random.randn() # np.random.randn(1) 1有增加array维度的作用    
print(action)


[1.05185142]


array([-0.34829525, -1.38448912])

In [67]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class PolicyNet(torch.nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim=128,):
        super(PolicyNet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, action_dim)
        '''
        #根据计算增益
        gain1 = nn.init.calculate_gain('relu')
        gain2 = nn.init.calculate_gain('tanh')
        #Xavier均匀分布初始化
        torch.nn.init.xavier_uniform_(self.fc1.weight, gain=gain1)
        torch.nn.init.xavier_uniform_(self.fc2.weight, gain=gain2)
        #初始化参数
        #self.fc1.weight.data.normal_(0, 0.1)
        self.fc1.bias.data.fill_(0.01)
        #self.fc2.weight.data.normal_(0, 0.1)
        self.fc2.bias.data.fill_(0.01)

        #self.action_bound = action_bound  # action_bound是环境可以接受的动作最大值
        '''
    def forward(self, x):
        x = F.relu(self.fc1(x))
        return torch.tanh(self.fc2(x)) # tanh输出范围是(-1,1)，符合动作的范围 还需要后续clip
policy = PolicyNet(12,1)
obs = torch.tensor([1,2,3,4,5,6,7,8,9,10,11,12],dtype=torch.float32)
print(policy(obs))
torch.randn(1) 

tensor([-0.7499], grad_fn=<TanhBackward0>)


tensor([-0.0958])

In [73]:
import numpy as np
win_list = [1,0,0,0,0,0,0,0,0,0]
win_array = np.mean(np.array(win_list).reshape(-1, 10), axis=1) #axis=1表示按行求均值
print(win_array)
print(np.array(win_list).reshape(-1, 10))

[0.1]
[[1 0 0 0 0 0 0 0 0 0]]


In [2]:
done ={'Red-0': False, 'Red-1': False, 'Red-2': False}
print(any(done.values()))

False


In [3]:
import pickle

# 假设你有一个名为 'data.pkl' 的文件
with open('results\\toy_env\\24\\rewards.pkl', 'rb') as file:
    data = pickle.load(file)

# 现在 'data' 变量包含了从 .pkl 文件中恢复的对象
print(data)


{'rewards': {'Red-0': array([ -4.        ,  -6.64236263,  -6.39353434, ..., 183.4901285 ,
       184.23064647, 188.91      ]), 'Red-1': array([128.31      ,  -5.24634945,  -6.43049854, ..., 185.66645085,
       194.87857453, 166.50867731]), 'Red-2': array([ -4.        ,  -4.02393492,  -8.36051615, ..., 187.9       ,
        14.18      , 187.9       ])}}


In [1]:
angle=[3.00596919]
assert 0 <= angle <= 2 * math.pi
print(angle)

TypeError: '<=' not supported between instances of 'int' and 'list'

In [7]:
import numpy as np
red0_hp_l = [0]
print(np.mean(red0_hp_l))

0.0


In [1]:
1e-3

0.001

In [1]:
4 // 4


1